In [ ]:
# Importing required libraries
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler, PCA
from pyspark.ml.clustering import GaussianMixture, KMeans
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, DoubleType
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

# Start Spark session
spark = SparkSession.builder.appName("TrafficDataML_Refinement").getOrCreate()

# Disable Arrow Optimization in Spark to prevent issues with VectorUDT conversion
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

# Load data
input_path = "dbfs:/user/mehak/processed/berlin_clean.csv"
spark_df = spark.read.csv(input_path, header=True, inferSchema=True)

# Show data and schema
spark_df.show(5)
spark_df.printSchema()

# Define categorical and numeric columns
categorical_low = ["spatial_type"]  # One-Hot Encoding
categorical_high = ["name", "berlin_bez"]  # Label Encoding
numeric_cols = ["zahl_tvz", "vz_typ_no", "lor_prg"]  # Adjust based on your dataset

# Encoding Categorical Features
indexers_high = [StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid="keep") for col in categorical_high]
indexers_low = [StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid="keep") for col in categorical_low]
encoders_low = [OneHotEncoder(inputCol=col+"_index", outputCol=col+"_vec") for col in categorical_low]

# Assemble features
assembler_inputs = numeric_cols + [col+"_index" for col in categorical_high] + [col+"_vec" for col in categorical_low]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

# Scale features
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

# -------------------------------
# Step 1: Gaussian Mixture Model (GMM) for Clustering
# -------------------------------
gmm = GaussianMixture(k=5, featuresCol="scaledFeatures", predictionCol="prediction")  # Note: 'prediction' instead of 'cluster'

# Build pipeline for GMM
gmm_pipeline = Pipeline(stages=indexers_high + indexers_low + encoders_low + [assembler, scaler, gmm])

# Fit GMM model
gmm_model = gmm_pipeline.fit(spark_df)
gmm_result = gmm_model.transform(spark_df)

# Show the first 10 cluster assignments for GMM
gmm_result.select("name", "spatial_type", "prediction").show(10)  # Note: 'prediction' instead of 'cluster'

# -------------------------------
# Step 2: Evaluate GMM with Silhouette Score
# -------------------------------
evaluator = ClusteringEvaluator(predictionCol="prediction")  # Note: 'prediction' instead of 'cluster'
silhouette = evaluator.evaluate(gmm_result)
print(f"GMM Silhouette Score: {silhouette}")

# -------------------------------
# Step 3: 3D PCA for Visualization (Dimensionality Reduction)
# -------------------------------
# Convert vector columns (e.g., scaledFeatures) to array before plotting
to_array_udf = udf(lambda v: v.toArray().tolist(), ArrayType(DoubleType()))
cleaned_df = gmm_result.withColumn("pcaArray", to_array_udf("scaledFeatures"))

# Apply PCA to reduce scaled features to 3D
pca = PCA(k=3, inputCol="scaledFeatures", outputCol="pcaFeatures")
pca_model = pca.fit(cleaned_df)
pca_df = pca_model.transform(cleaned_df)

# Collect PCA data into Pandas for visualization
plot_df = pca_df.select("pcaFeatures", "prediction").rdd.map(lambda row: (row['pcaFeatures'], row['prediction'])).toDF(["pcaArray", "prediction"]).toPandas()

# Extract x, y, z from PCA array for 3D plotting
plot_df["x"] = plot_df["pcaArray"].apply(lambda x: float(x[0]))
plot_df["y"] = plot_df["pcaArray"].apply(lambda x: float(x[1]))
plot_df["z"] = plot_df["pcaArray"].apply(lambda x: float(x[2]))

# Plotting 3D PCA
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
for cluster_id in plot_df["prediction"].unique():  # Note: 'prediction' instead of 'cluster'
    subset = plot_df[plot_df["prediction"] == cluster_id]
    ax.scatter(subset["x"], subset["y"], subset["z"], label=f"Cluster {cluster_id}", alpha=0.6)
ax.set_xlabel('PCA Feature 1')
ax.set_ylabel('PCA Feature 2')
ax.set_zlabel('PCA Feature 3')
ax.set_title("GMM Clusters (3D PCA Projection)")
ax.legend()
plt.show()

# -------------------------------
# Step 4: Elbow Method for Optimal k (Selecting k based on WSSSE for GMM)
# -------------------------------
wssse_list = []
k_range = range(2, 11)  # Testing k from 2 to 10 clusters for GMM
for k_val in k_range:
    gmm.setK(k_val)  # Set k value
    gmm_model = gmm_pipeline.fit(spark_df)  # Fit model with new k
    gmm_result = gmm_model.transform(spark_df)  # Apply to data
    evaluator = ClusteringEvaluator(predictionCol="prediction")  # Note: 'prediction' instead of 'cluster'
    wssse_list.append(evaluator.evaluate(gmm_result))  # Collect WSSSE for each k

# Plot Elbow Method to visualize the best k for GMM
plt.figure(figsize=(8, 6))
plt.plot(k_range, wssse_list, marker='o')
plt.title("Elbow Method for Optimal k (GMM)")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("WSSSE")
plt.xticks(k_range)
plt.grid(True)
plt.show()

# -------------------------------
# Step 5: Cluster Analysis (Mean Values of Features by Cluster)
# -------------------------------
# Compute the mean values for each feature per cluster
cluster_summary = gmm_result.groupBy("prediction").mean("zahl_tvz", "vz_typ_no", "lor_prg")  # Note: 'prediction' instead of 'cluster'
cluster_summary.show()

# -------------------------------
# Step 6: Visualize Cluster Characteristics (Traffic Volume and Vehicle Type Distribution)
# -------------------------------
# Plot the distribution of traffic volume by cluster
sns.boxplot(x="prediction", y="zahl_tvz", data=gmm_result.toPandas())  # Note: 'prediction' instead of 'cluster'
plt.title("Traffic Volume Distribution by Cluster (GMM)")
plt.show()

# Visualize the distribution of vehicle type by cluster
sns.boxplot(x="prediction", y="vz_typ_no", data=gmm_result.toPandas())  # Note: 'prediction' instead of 'cluster'
plt.title("Vehicle Type Distribution by Cluster (GMM)")
plt.show()
